In [7]:
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
from flightsql import FlightSQLClient

# Read only token for demo purposes
token = "6mFPNdcEwrjQD9utxkkMS6BfmhJoMIYsHkI317EcGSCMZaTalYADf0zm6u4VqrBv5YiGvvOf5Qa8sYTVrDigeA=="
 
client = FlightSQLClient(host='eu-central-1-1.aws.cloud2.influxdata.com',
                        token=token,
                        metadata={'bucket-name': 'factory'},
                        features={'metadata-reflection': 'true'})

# Execute a query against InfluxDB's Flight SQL endpoint                         
#query = client.execute("SELECT * FROM iox.mqtt_consumer WHERE time > (NOW() - INTERVAL '1 DAY') ")

query = client.execute("SELECT * FROM iox.machine_data")

# Create reader to consume result
reader = client.do_get(query.endpoints[0].ticket)

# Read all data into a pyarrow.Table
table = reader.read_all()
print(table)

pyarrow.Table
host: string
load: double
machineID: string
power: double
provider: string
temperature: double
time: timestamp[ns] not null
topic: string
vibration: double
----
host: [["285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb",...,"285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb"],["285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb",...,"285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb"],...,["6e8a413a19a3","6e8a413a19a3","6e8a413a19a3","6e8a413a19a3","6e8a413a19a3",...,"6e8a413a19a3","6e8a413a19a3","6e8a413a19a3","6e8a413a19a3","6e8a413a19a3"],["6e8a413a19a3","6e8a413a19a3","6e8a413a19a3","6e8a413a19a3","6e8a413a19a3",...,"6e8a413a19a3","6e8a413a19a3","6e8a413a19a3","6e8a413a19a3","6e8a413a19a3"]]
load: [[50,50,50,50,50,...,55,55,55,55,55],[55,55,5,5,5,...,37,37,37,37,37],...,[120,120,120,120,120,...,1,1,1,1,1],[42,42,42,42,42,...,58,58,58,58,58]]
machineID: [["machine1","machine1","

In [12]:
# Save the table to a Parquet file
pq.write_table(table, 'large.parquet', version='2.6')

In [13]:
# Load the table from the Parquet file
table2 = pq.read_table('large.parquet')
# table2 = table2.to_pandas()
print(table2)

pyarrow.Table
host: string
load: double
machineID: string
power: double
provider: string
temperature: double
time: timestamp[ns] not null
topic: string
vibration: double
----
host: [["285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb","285b7993efcb",...,"6e8a413a19a3","6e8a413a19a3","6e8a413a19a3","6e8a413a19a3","6e8a413a19a3"],["6e8a413a19a3","6e8a413a19a3","6e8a413a19a3","6e8a413a19a3","6e8a413a19a3",...,"6e8a413a19a3","6e8a413a19a3","6e8a413a19a3","6e8a413a19a3","6e8a413a19a3"],...,["6e8a413a19a3","6e8a413a19a3","6e8a413a19a3","6e8a413a19a3","6e8a413a19a3",...,"6e8a413a19a3","6e8a413a19a3","6e8a413a19a3","6e8a413a19a3","6e8a413a19a3"],["6e8a413a19a3","6e8a413a19a3","6e8a413a19a3","6e8a413a19a3","6e8a413a19a3",...,"6e8a413a19a3","6e8a413a19a3","6e8a413a19a3","6e8a413a19a3","6e8a413a19a3"]]
load: [[50,50,50,50,50,...,20,20,20,20,59],[59,59,59,59,59,...,15,15,15,15,15],...,[20,20,20,20,20,...,6,6,6,6,6],[6,6,6,6,6,...,58,58,58,58,58]]
machineID: [["machine1","machine1","machine

In [10]:
dataframe = pd.read_parquet('large.parquet')
print(dataframe)

                host  load machineID  power                  provider  \
0       285b7993efcb  50.0  machine1  220.0  Arroyo, Collier and Hall   
1       285b7993efcb  50.0  machine1  205.0  Arroyo, Collier and Hall   
2       285b7993efcb  50.0  machine1  213.0  Arroyo, Collier and Hall   
3       285b7993efcb  50.0  machine1  213.0  Arroyo, Collier and Hall   
4       285b7993efcb  50.0  machine1  211.0  Arroyo, Collier and Hall   
...              ...   ...       ...    ...                       ...   
565066  6e8a413a19a3  58.0  machine3  209.0             Wells-Sanchez   
565067  6e8a413a19a3  58.0  machine3  214.0             Wells-Sanchez   
565068  6e8a413a19a3  58.0  machine3  207.0             Wells-Sanchez   
565069  6e8a413a19a3  58.0  machine3  208.0             Wells-Sanchez   
565070  6e8a413a19a3  58.0  machine3  209.0             Wells-Sanchez   

        temperature                          time             topic  vibration  
0              39.0 2023-03-22 11:19:28.02